In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np

#-------------- NESTED PATH CORRECTION --------------------------------#

# For all script files, we add the parent directory to the system path
cwd = re.sub(r"[\\]", "/", os.getcwd())
cwd_list = cwd.split("/")
path = sys.argv[0]
path_list = path.split("/")
# either the entire filepath is entered as command i python
if cwd_list[0:3] == path_list[0:3]:
    full_path = path
# or a relative path is entered, in which case we append the path to the cwd_path
else:
    full_path = cwd + "/" + path
# remove the overlap
root_dir = re.search(r"(^.+python_road_upgrades)", full_path).group(1)
sys.path.append(root_dir)

#----------------------------------------------------------------------#

def dummy_row_distance(df, dummy, agg_list):
    if not isinstance(agg_list, list):
        raise ValueError("agg_list argument needs to be a list")
    output_index = df.reset_index().set_index([('road', ''), ('subroad', '')])
    filtered = df.loc[df[dummy] == 1,]
    dist = filtered.reset_index().loc[:, [('road', ''), ('subroad', ''), ('id', '')]].groupby(['road', 'subroad']).diff().fillna(0).values
    dist = pd.DataFrame(dist, index=filtered.index).reset_index().rename(columns={0: "dist"})
    dist = dist.reset_index()#.group_by(['road', 'subroad'])
    dist = dist.groupby(['road', 'subroad']).agg({'dist': agg_list})
    dist = dist.reindex(df.index.droplevel(2).unique())
    return dist

folder_name = "composites_2"        # directory should exist in Exports/Roads and Exports/Points


dir_roads = f"{root_dir}/Exports/Roads/{folder_name}"
dir_points = f"{root_dir}/Exports/Points/{folder_name}"

files_roads = os.listdir(dir_roads)
files_points = os.listdir(dir_points)

In [2]:
for r in files_roads:
    # r = "roads_2020_2021_05_07.xlsx"
    if r[0:5] != 'roads':
        continue
    m = re.search(r"roads_(\d+)_(\d+)_(\d+)_(\d+).xlsx", r)
    year1, year2, month1, month2 = m.group(1), m.group(2), m.group(3), m.group(4)

    # load in the roads excel file
    fp_r = f"{dir_roads}/{r}"
    df_r = pd.read_excel(fp_r, engine='openpyxl')

    # load in the csv points files and merge
    df_p = pd.DataFrame()
    for p in files_points:
        if r[:-5] in p:
            fp_p = f"{dir_points}/{p}"
            imported = pd.read_csv(fp_p)
            if df_p.empty:
                df_p = imported
            else:
                df_p = pd.concat([df_p, imported])
    df_p = df_p.sort_values(by=["road", "subroad", "type", "id"])
    break

df_p

,road,subroad,type,id,coords,B4_max,B4_mean,B4_min,B8_max,B8_mean,B8_min,B4_diff_factor,B8_diff_factor
179,0,0,left_25,0,"[40.09473727528661, -14.093615299623254]",330.0,228.272962,399.0,-171.0,-143.633894,-115.0,87.897659,-80.030474
180,0,0,left_25,1,"[40.094849849313206, -14.093757186756775]",204.0,212.598456,174.0,-257.0,-268.265122,-339.0,87.897659,-80.030474
181,0,0,left_25,2,"[40.0949624233398, -14.093899073890295]",62.0,159.989597,232.0,-269.0,-321.681404,-381.0,87.897659,-80.030474
182,0,0,left_25,3,"[40.095074997366396, -14.094040961023815]",2196.0,1261.910968,987.0,737.0,146.529032,-217.0,87.897659,-80.030474
183,0,0,left_25,4,"[40.09518757139299, -14.094182848157336]",-132.0,192.713178,179.0,76.0,-119.275194,-360.0,87.897659,-80.030474
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121053,218,0,road,1500,"[37.48840834097069, -17.01079496508643]",457.5,285.161765,248.5,272.5,308.169437,395.0,3.550247,143.710757
110533,218,0,road,1501,"[37.48836932383648, -17.0106589625043]",626.5,381.546556,319.5,581.5,348.328444,280.5,3.550247,143.710757
121054,218,0,road,1501,"[37.48836932383648, -17.0106589625043]",626.5,381.546556,319.5,581.5,348.328444,280.5,3.550247,143.710757
110534,218,0,road,1502,"[37.48833030670226, -17.010522959922174]",9.0,114.116561,40.5,259.0,175.812739,31.0,3.550247,143.710757
